Use census data to predict income as greater or less than 50,000. This is a binary classification problem.

In [1]:
import functools, inspect, os, sys, tempfile

import pandas, requests

import matplotlib.pyplot as plt

import numpy as np

import tensorflow as tf
import tensorflow.feature_column as fc

In [2]:
tf.enable_eager_execution()

## Official Implementation

From tensorflow model repository.

In [3]:
! git clone --depth 1 https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


In [4]:
models_path = os.path.join(os.getcwd(), 'models')
models_path

'/home/ec2-user/SageMaker/ml-production-scale/models'

In [5]:
sys.path.append(models_path)

In [6]:
from official.wide_deep import census_dataset, census_main

In [7]:
census_dataset.download('/tmp/census_data/')

### Command Line

In [8]:
if "PYTHONPATH" in os.environ:
    os.environ['PYTHONPATH'] += os.pathsep +  models_path
else:
    os.environ['PYTHONPATH'] = models_path

In [9]:
!python -m official.wide_deep.census_main --help

Train DNN on census income dataset.
flags:

/home/ec2-user/SageMaker/ml-production-scale/models/official/wide_deep/census_main.py:
  -bs,--batch_size:
    Batch size for training and evaluation. When using multiple gpus, this is
    the
    global batch size for all devices. For example, if the batch size is 32 and
    there are 4 GPUs, each GPU will get 8 examples on each step.
    (default: '40')
    (an integer)
  --[no]clean:
    If set, model_dir will be removed if it exists.
    (default: 'false')
  -dd,--data_dir:
    The location of the input data.
    (default: '/tmp/census_data')
  --[no]download_if_missing:
    Download data to data_dir if it is not already present.
    (default: 'true')
  -ebe,--epochs_between_evals:
    The number of training epochs to run between evaluations.
    (default: '2')
    (an integer)
  -ed,--export_dir:
    If set, a SavedModel serialization of the model will be exported to this
    directory at the end of training. See the README for more deta

Run model from command line.

In [10]:
!python -m official.wide_deep.census_main --model_type=wide --train_epochs=2

I0105 04:19:45.093888 140511467317056 tf_logging.py:115] Using config: {'_model_dir': '/tmp/census_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': device_count {
  key: "GPU"
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcb29e3f208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
W0105 04:19:45.094872 140511467317056 tf_logging.py:120] 'cpuinfo' not imported. CPU info will not be logged.
I0105 04:19:45.143214 140511467317056 tf_logging.py:115] Benchmark run: {'model_name': 'wide_deep', 'dataset': {'name': 'Census Income'}, 'machine_config': {'gpu_info': {'count': 0

I0105 04:19:59.196825 140511467317056 tf_logging.py:115] Running local_init_op.
I0105 04:19:59.252202 140511467317056 tf_logging.py:115] Done running local_init_op.
I0105 04:20:02.049376 140511467317056 tf_logging.py:115] Finished evaluation at 2019-01-05-04:20:02
I0105 04:20:02.049627 140511467317056 tf_logging.py:115] Saving dict for global step 4887: accuracy = 0.8352067, accuracy_baseline = 0.76377374, auc = 0.88247764, auc_precision_recall = 0.6936035, average_loss = 0.3532486, global_step = 4887, label/mean = 0.23622628, loss = 14.096177, precision = 0.6806462, prediction/mean = 0.24159718, recall = 0.5696828
I0105 04:20:02.334942 140511467317056 tf_logging.py:115] Saving 'checkpoint_path' summary for global step 4887: /tmp/census_model/model.ckpt-4887
I0105 04:20:02.335492 140511467317056 tf_logging.py:115] Results at epoch 2 / 2
I0105 04:20:02.335591 140511467317056 tf_logging.py:115] ------------------------------------------------------------
I0105 04:20:02.335675 14051146731

## Read Census Data

In [11]:
!ls /tmp/census_data/

adult.data  adult.test


In [12]:
train_file = '/tmp/census_data/adult.data'
test_file = '/tmp/census_data/adult.test'

In [13]:
train_df = pandas.read_csv(train_file, header=None, names=census_dataset._CSV_COLUMNS)
test_df = pandas.read_csv(test_file, header=None, names=census_dataset._CSV_COLUMNS)

train_df.sample(5)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
10917,30,Self-emp-inc,119422,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,70,United-States,<=50K
23320,56,Local-gov,52953,Doctorate,16,Divorced,Prof-specialty,Not-in-family,White,Female,0,1669,38,United-States,<=50K
9642,62,Private,118696,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
13281,26,Private,243560,Some-college,10,Never-married,Sales,Unmarried,White,Female,0,0,40,?,<=50K
3063,36,Private,140583,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


## Convert Data to Tensors

Create input function.

In [14]:
# input function to go from pandas dataframe to tensorflow dataset
def easy_input_function(df, label_key, num_epochs, shuffle, batch_size):
    
    label = df[label_key]
    ds = tf.data.Dataset.from_tensor_slices((dict(df), label))
    
    if shuffle:
        ds = ds.shuffle(10000)
        
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds

ds = easy_input_function(train_df, label_key='income_bracket', num_epochs=5, shuffle=True, batch_size=10)
ds

<RepeatDataset shapes: ({age: (?,), workclass: (?,), fnlwgt: (?,), education: (?,), education_num: (?,), marital_status: (?,), occupation: (?,), relationship: (?,), race: (?,), gender: (?,), capital_gain: (?,), capital_loss: (?,), hours_per_week: (?,), native_country: (?,), income_bracket: (?,)}, (?,)), types: ({age: tf.int32, workclass: tf.string, fnlwgt: tf.int32, education: tf.string, education_num: tf.int32, marital_status: tf.string, occupation: tf.string, relationship: tf.string, race: tf.string, gender: tf.string, capital_gain: tf.int32, capital_loss: tf.int32, hours_per_week: tf.int32, native_country: tf.string, income_bracket: tf.string}, tf.string)>

In [15]:
# inspect
for feature_batch, label_batch in ds.take(1):
    print(list(feature_batch.keys()))
    print(feature_batch['age'])
    print(label_batch)

['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'gender', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income_bracket']
tf.Tensor([18 53 25 36 45 21 37 31 44 67], shape=(10,), dtype=int32)
tf.Tensor(
[b'<=50K' b'<=50K' b'<=50K' b'>50K' b'<=50K' b'<=50K' b'<=50K' b'<=50K'
 b'>50K' b'>50K'], shape=(10,), dtype=string)


Large datasets should be streamed from disk.

In [16]:
# input function from census_dataset
print(inspect.getsource(census_dataset.input_fn))

def input_fn(data_file, num_epochs, shuffle, batch_size):
  """Generate an input function for the Estimator."""
  assert tf.gfile.Exists(data_file), (
      '%s not found. Please make sure you have run census_dataset.py and '
      'set the --data_dir argument to the correct path.' % data_file)

  def parse_csv(value):
    tf.logging.info('Parsing {}'.format(data_file))
    columns = tf.decode_csv(value, record_defaults=_CSV_COLUMN_DEFAULTS)
    features = dict(zip(_CSV_COLUMNS, columns))
    labels = features.pop('income_bracket')
    classes = tf.equal(labels, '>50K')  # binary classification
    return features, classes

  # Extract lines from input files using the Dataset API.
  dataset = tf.data.TextLineDataset(data_file)

  if shuffle:
    dataset = dataset.shuffle(buffer_size=_NUM_EXAMPLES['train'])

  dataset = dataset.map(parse_csv, num_parallel_calls=5)

  # We call repeat after shuffling, rather than before, to prevent separate
  # epochs from blending together.
  dataset = 

In [17]:
# census_dataset.input_fn produces compatible output to the custom one above
ds = census_dataset.input_fn(train_file, batch_size=10, num_epochs=5, shuffle=True)
for feature_batch, label_batch in ds.take(1):
    print(list(feature_batch.keys()))
    print(feature_batch['age'])
    print(label_batch)

INFO:tensorflow:Parsing /tmp/census_data/adult.data


I0105 04:20:03.189923 140042238199616 tf_logging.py:115] Parsing /tmp/census_data/adult.data


['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'gender', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country']
tf.Tensor([36 36 48 42 26 62 57 28 36 20], shape=(10,), dtype=int32)
tf.Tensor([ True False False False False False  True False False False], shape=(10,), dtype=bool)


Estimators expect input functions that take no arguments.

In [18]:
train_input_fn = functools.partial(census_dataset.input_fn, train_file, batch_size=64, num_epochs=2, shuffle=True)
test_input_fn = functools.partial(census_dataset.input_fn, train_file, batch_size=64, num_epochs=1, shuffle=False)

train_input_fn

functools.partial(<function input_fn at 0x7f5dc85172f0>, '/tmp/census_data/adult.data', batch_size=64, num_epochs=2, shuffle=True)

## Selecting and Engineering Features

### Base Feature Columns

#### Numeric Columns

In [19]:
age = fc.numeric_column("age")
age

_NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

In [20]:
fc.input_layer(feature_batch, [age]).numpy()

array([[36.],
       [36.],
       [48.],
       [42.],
       [26.],
       [62.],
       [57.],
       [28.],
       [36.],
       [20.]], dtype=float32)

Evaluate model using only the `age` feature.

In [21]:
classifier = tf.estimator.LinearClassifier(feature_columns=[age])
classifier

INFO:tensorflow:Using default config.


I0105 04:20:03.321693 140042238199616 tf_logging.py:115] Using default config.


W0105 04:20:03.325370 140042238199616 tf_logging.py:125] Using temporary folder as model directory: /tmp/tmp_jt2ryvu


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_jt2ryvu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5dc85059b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0105 04:20:03.327496 140042238199616 tf_logging.py:115] Using config: {'_model_dir': '/tmp/tmp_jt2ryvu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5dc85059b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [22]:
classifier.train(train_input_fn)
result = classifier.evaluate(test_input_fn)

INFO:tensorflow:Parsing /tmp/census_data/adult.data


I0105 04:20:03.353101 140042238199616 tf_logging.py:115] Parsing /tmp/census_data/adult.data


INFO:tensorflow:Calling model_fn.


I0105 04:20:03.388395 140042238199616 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Done calling model_fn.


...

I0105 04:20:12.904404 140042238199616 tf_logging.py:115] Finished evaluation at 2019-01-05-04:20:12


INFO:tensorflow:Saving dict for global step 1018: accuracy = 0.73511255, accuracy_baseline = 0.75919044, auc = 0.68424314, auc_precision_recall = 0.3220013, average_loss = 0.5313502, global_step = 1018, label/mean = 0.24080956, loss = 33.990753, precision = 0.20658682, prediction/mean = 0.28833595, recall = 0.03519959


I0105 04:20:12.908713 140042238199616 tf_logging.py:115] Saving dict for global step 1018: accuracy = 0.73511255, accuracy_baseline = 0.75919044, auc = 0.68424314, auc_precision_recall = 0.3220013, average_loss = 0.5313502, global_step = 1018, label/mean = 0.24080956, loss = 33.990753, precision = 0.20658682, prediction/mean = 0.28833595, recall = 0.03519959


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1018: /tmp/tmp_jt2ryvu/model.ckpt-1018


I0105 04:20:13.076347 140042238199616 tf_logging.py:115] Saving 'checkpoint_path' summary for global step 1018: /tmp/tmp_jt2ryvu/model.ckpt-1018


In [23]:
result

{'accuracy': 0.73511255,
 'accuracy_baseline': 0.75919044,
 'auc': 0.68424314,
 'auc_precision_recall': 0.3220013,
 'average_loss': 0.5313502,
 'label/mean': 0.24080956,
 'loss': 33.990753,
 'precision': 0.20658682,
 'prediction/mean': 0.28833595,
 'recall': 0.03519959,
 'global_step': 1018}

In [24]:
# define numeric columns
educated_num = fc.numeric_column('education_num')
capital_gain = fc.numeric_column('capital_gain')
capital_loss = fc.numeric_column('capital_loss')
hours_per_week = fc.numeric_column('hours_per_week')

numeric_columns = [age, educated_num, capital_gain, capital_loss, hours_per_week]
fc.input_layer(feature_batch, numeric_columns).numpy()

array([[  36.,    0.,    0.,    3.,   50.],
       [  36.,    0.,    0.,   13.,   70.],
       [  48.,    0.,    0.,    9.,   40.],
       [  42.,    0.,    0.,   10.,   23.],
       [  26.,    0., 1602.,    9.,   15.],
       [  62.,    0.,    0.,    9.,   35.],
       [  57.,    0.,    0.,    9.,   50.],
       [  28.,    0.,    0.,   11.,   40.],
       [  36.,    0., 1672.,   13.,   50.],
       [  20.,    0.,    0.,   10.,   15.]], dtype=float32)

Evaluate model on these numeric columns.

In [26]:
classifier = tf.estimator.LinearClassifier(feature_columns=numeric_columns)
classifier.train(train_input_fn)
result = classifier.evaluate(test_input_fn)

INFO:tensorflow:Using default config.


I0105 04:20:55.320209 140042238199616 tf_logging.py:115] Using default config.


W0105 04:20:55.322726 140042238199616 tf_logging.py:125] Using temporary folder as model directory: /tmp/tmpswnfo6ca


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpswnfo6ca', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5db01f6b70>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0105 04:20:55.324474 140042238199616 tf_logging.py:115] Using config: {'_model_dir': '/tmp/tmpswnfo6ca', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5db01f6b70>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


...

I0105 04:21:06.888761 140042238199616 tf_logging.py:115] Finished evaluation at 2019-01-05-04:21:06


INFO:tensorflow:Saving dict for global step 1018: accuracy = 0.7800743, accuracy_baseline = 0.75919044, auc = 0.78540415, auc_precision_recall = 0.55917776, average_loss = 2.589815, global_step = 1018, label/mean = 0.24080956, loss = 165.67183, precision = 0.58056873, prediction/mean = 0.29486868, recall = 0.31246015


I0105 04:21:06.892976 140042238199616 tf_logging.py:115] Saving dict for global step 1018: accuracy = 0.7800743, accuracy_baseline = 0.75919044, auc = 0.78540415, auc_precision_recall = 0.55917776, average_loss = 2.589815, global_step = 1018, label/mean = 0.24080956, loss = 165.67183, precision = 0.58056873, prediction/mean = 0.29486868, recall = 0.31246015


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1018: /tmp/tmpswnfo6ca/model.ckpt-1018


I0105 04:21:07.078623 140042238199616 tf_logging.py:115] Saving 'checkpoint_path' summary for global step 1018: /tmp/tmpswnfo6ca/model.ckpt-1018


In [27]:
result

{'accuracy': 0.7800743,
 'accuracy_baseline': 0.75919044,
 'auc': 0.78540415,
 'auc_precision_recall': 0.55917776,
 'average_loss': 2.589815,
 'label/mean': 0.24080956,
 'loss': 165.67183,
 'precision': 0.58056873,
 'prediction/mean': 0.29486868,
 'recall': 0.31246015,
 'global_step': 1018}

#### Categorical Columns

In [28]:
# create sparse one-hot vector, values known in advance
relationship = fc.categorical_column_with_vocabulary_list("relationship",
    ['Husband', 'Not-in-family', 'Wife', 'Own-child', 'Unmarried', 'Other-relative'])
fc.input_layer(feature_batch, [age, fc.indicator_column(relationship)])

<tf.Tensor: id=5375, shape=(10, 7), dtype=float32, numpy=
array([[36.,  1.,  0.,  0.,  0.,  0.,  0.],
       [36.,  0.,  1.,  0.,  0.,  0.,  0.],
       [48.,  0.,  0.,  0.,  1.,  0.,  0.],
       [42.,  0.,  0.,  1.,  0.,  0.,  0.],
       [26.,  0.,  0.,  0.,  0.,  0.,  1.],
       [62.,  1.,  0.,  0.,  0.,  0.,  0.],
       [57.,  1.,  0.,  0.,  0.,  0.,  0.],
       [28.,  0.,  0.,  0.,  1.,  0.,  0.],
       [36.,  1.,  0.,  0.,  0.,  0.,  0.],
       [20.,  0.,  0.,  0.,  1.,  0.,  0.]], dtype=float32)>

In [29]:
# values unknown
occupation = fc.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
for item in feature_batch['occupation'].numpy():
    print(item.decode())

Transport-moving
Exec-managerial
Sales
Other-service
Priv-house-serv
Farming-fishing
Craft-repair
Exec-managerial
Prof-specialty
Adm-clerical


In [30]:
# look at result from hash bucket categorical column
occupation_result = fc.input_layer(feature_batch, [fc.indicator_column(occupation)])
occupation_result.numpy().shape

(10, 1000)

In [31]:
tf.argmax(occupation_result, axis=1)

<tf.Tensor: id=5401, shape=(10,), dtype=int64, numpy=array([420, 800, 631, 527, 250, 936, 466, 800, 979,  96])>

In [32]:
education = fc.categorical_column_with_vocabulary_list(
    'education', [
        'Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college',
        'Assoc-acdm', 'Assoc-voc', '7th-8th', 'Doctorate', 'Prof-school',
        '5th-6th', '10th', '1st-4th', 'Preschool', '12th'])

marital_status = fc.categorical_column_with_vocabulary_list(
    'marital_status', [
        'Married-civ-spouse', 'Divorced', 'Married-spouse-absent',
        'Never-married', 'Separated', 'Married-AF-spouse', 'Widowed'])

workclass = fc.categorical_column_with_vocabulary_list(
    'workclass', [
        'Self-emp-not-inc', 'Private', 'State-gov', 'Federal-gov',
        'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked'])

categorical_columns = [relationship, occupation, education, marital_status, workclass]
all_columns = numeric_columns + categorical_columns

all_columns

[_NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='education_num', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='capital_gain', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='capital_loss', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='hours_per_week', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _VocabularyListCategoricalColumn(key='relationship', vocabulary_list=('Husband', 'Not-in-family', 'Wife', 'Own-child', 'Unmarried', 'Other-relative'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 _HashedCategoricalColumn(key='occupation', hash_bucket_size=1000, dtype=tf.string),
 _VocabularyListCategoricalColumn(key='education', vocabulary_list=('Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college', 'Assoc-acdm', 'Assoc-voc', '7th-8th', 'Doctor

Train on all the features.

In [33]:
classifier = tf.estimator.LinearClassifier(feature_columns=all_columns)
classifier.train(train_input_fn)
result = classifier.evaluate(test_input_function)

INFO:tensorflow:Using default config.


I0105 04:21:14.186759 140042238199616 tf_logging.py:115] Using default config.


W0105 04:21:14.189560 140042238199616 tf_logging.py:125] Using temporary folder as model directory: /tmp/tmprburu1pt


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmprburu1pt', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5db2c8c320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0105 04:21:14.191135 140042238199616 tf_logging.py:115] Using config: {'_model_dir': '/tmp/tmprburu1pt', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5db2c8c320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


...

INFO:tensorflow:Saving checkpoints for 1018 into /tmp/tmprburu1pt/model.ckpt.


I0105 04:21:23.643472 140042238199616 tf_logging.py:115] Saving checkpoints for 1018 into /tmp/tmprburu1pt/model.ckpt.


INFO:tensorflow:Loss for final step: 12.291178.


I0105 04:21:23.747963 140042238199616 tf_logging.py:115] Loss for final step: 12.291178.


NameError: name 'test_input_function' is not defined

In [34]:
result

{'accuracy': 0.7800743,
 'accuracy_baseline': 0.75919044,
 'auc': 0.78540415,
 'auc_precision_recall': 0.55917776,
 'average_loss': 2.589815,
 'label/mean': 0.24080956,
 'loss': 165.67183,
 'precision': 0.58056873,
 'prediction/mean': 0.29486868,
 'recall': 0.31246015,
 'global_step': 1018}

### Derived Features

#### Bucketization

Continuous features can be made categorical via bucketization.

In [35]:
age_buckets = fc.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
fc.input_layer(feature_batch, [age, age_buckets]).numpy()

array([[36.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [36.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [48.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [42.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [26.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [62.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [57.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [28.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [36.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [20.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],
      dtype=float32)

#### Crossed Column

In [36]:
education_x_occupation = fc.crossed_column(["education", "occupation"], hash_bucket_size=1000)
age_buckets_x_education_x_occupation = fc.crossed_column([age_buckets, "education", "occupation"], hash_bucket_size=1000)

## Logistic Regression Model

In [37]:
base_columns = [education, marital_status, relationship, workclass, occupation, age_buckets]

crossed_columns = [education_x_occupation, age_buckets_x_education_x_occupation]
all_columns = base_columns + crossed_columns

all_columns

[_VocabularyListCategoricalColumn(key='education', vocabulary_list=('Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college', 'Assoc-acdm', 'Assoc-voc', '7th-8th', 'Doctorate', 'Prof-school', '5th-6th', '10th', '1st-4th', 'Preschool', '12th'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 _VocabularyListCategoricalColumn(key='marital_status', vocabulary_list=('Married-civ-spouse', 'Divorced', 'Married-spouse-absent', 'Never-married', 'Separated', 'Married-AF-spouse', 'Widowed'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 _VocabularyListCategoricalColumn(key='relationship', vocabulary_list=('Husband', 'Not-in-family', 'Wife', 'Own-child', 'Unmarried', 'Other-relative'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 _VocabularyListCategoricalColumn(key='workclass', vocabulary_list=('Self-emp-not-inc', 'Private', 'State-gov', 'Federal-gov', 'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked'), dtype=tf.string, default_value=-1, num_oov_

In [38]:
model = tf.estimator.LinearClassifier(feature_columns=all_columns, model_dir=tempfile.mkdtemp(),
    optimizer=tf.train.FtrlOptimizer(learning_rate=0.1))
model

INFO:tensorflow:Using default config.


I0105 04:21:40.567739 140042238199616 tf_logging.py:115] Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp67wh6vbr', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5db00364a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0105 04:21:40.569427 140042238199616 tf_logging.py:115] Using config: {'_model_dir': '/tmp/tmp67wh6vbr', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5db00364a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Train / Evaluate Model

In [39]:
train_input_fn = functools.partial(census_dataset.input_fn, train_file, batch_size=64, num_epochs=40, shuffle=True)
train_input_fn

functools.partial(<function input_fn at 0x7f5dc85172f0>, '/tmp/census_data/adult.data', batch_size=64, num_epochs=40, shuffle=True)

In [40]:
model.train(train_input_fn)

INFO:tensorflow:Parsing /tmp/census_data/adult.data


I0105 04:21:45.554280 140042238199616 tf_logging.py:115] Parsing /tmp/census_data/adult.data


INFO:tensorflow:Calling model_fn.


I0105 04:21:45.591936 140042238199616 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Done calling model_fn.


...

I0105 04:23:52.677122 140042238199616 tf_logging.py:115] loss = 21.137072, step = 20301 (0.564 sec)


INFO:tensorflow:Saving checkpoints for 20351 into /tmp/tmp67wh6vbr/model.ckpt.


I0105 04:23:52.945179 140042238199616 tf_logging.py:115] Saving checkpoints for 20351 into /tmp/tmp67wh6vbr/model.ckpt.


INFO:tensorflow:Loss for final step: 14.731519.


I0105 04:23:53.062778 140042238199616 tf_logging.py:115] Loss for final step: 14.731519.


In [41]:
results = model.evaluate(test_input_fn)

INFO:tensorflow:Parsing /tmp/census_data/adult.data


I0105 04:23:53.082465 140042238199616 tf_logging.py:115] Parsing /tmp/census_data/adult.data


INFO:tensorflow:Calling model_fn.


I0105 04:23:53.116172 140042238199616 tf_logging.py:115] Calling model_fn.


W0105 04:23:54.536241 140042238199616 tf_logging.py:125] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.


...

I0105 04:23:59.667019 140042238199616 tf_logging.py:115] Finished evaluation at 2019-01-05-04:23:59


INFO:tensorflow:Saving dict for global step 20351: accuracy = 0.844538, accuracy_baseline = 0.75919044, auc = 0.89675534, auc_precision_recall = 0.72774917, average_loss = 0.33607608, global_step = 20351, label/mean = 0.24080956, loss = 21.498966, precision = 0.71606284, prediction/mean = 0.2409244, recall = 0.58729756


I0105 04:23:59.671287 140042238199616 tf_logging.py:115] Saving dict for global step 20351: accuracy = 0.844538, accuracy_baseline = 0.75919044, auc = 0.89675534, auc_precision_recall = 0.72774917, average_loss = 0.33607608, global_step = 20351, label/mean = 0.24080956, loss = 21.498966, precision = 0.71606284, prediction/mean = 0.2409244, recall = 0.58729756


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20351: /tmp/tmp67wh6vbr/model.ckpt-20351


I0105 04:23:59.950124 140042238199616 tf_logging.py:115] Saving 'checkpoint_path' summary for global step 20351: /tmp/tmp67wh6vbr/model.ckpt-20351


In [42]:
results

{'accuracy': 0.844538,
 'accuracy_baseline': 0.75919044,
 'auc': 0.89675534,
 'auc_precision_recall': 0.72774917,
 'average_loss': 0.33607608,
 'label/mean': 0.24080956,
 'loss': 21.498966,
 'precision': 0.71606284,
 'prediction/mean': 0.2409244,
 'recall': 0.58729756,
 'global_step': 20351}

In [ ]:
predict_df = test_df.sample(20).copy()
predict_df

In [ ]:
predict_iter = model.predict(lambda:easy_input_function(predict_df, batch_size=10,
    label_key="income_bracket", num_epochs=1, shuffle=False))

In [ ]:
classes = np.array(['<=50K', '>50K'])
pred_class_id = []

In [ ]:
for pred_dict in predict_iter:
    pred_class_id.append(pred_dict["class_ids"])
    
predict_df['predicted_class'] = classes[np.array(pred_class_id)]
predict_df['correct'] = predict_df['predicted_class'] == predict_df['income_bracket']

In [ ]:
predict_df[['income_bracket','predicted_class', 'correct']]

## Regularization

In [ ]:
model_l1 = tf.estimator.LinearClassifier(feature_columns=all_columns, optimizer=tf.train.FtrlOptimizer(
    l1_regularization_strength=10.0, l2_regularization_strength=0.0, learning_rate=0.1))
model_l1

In [ ]:
model_l1.train(train_input_fn)

In [ ]:
results = model_l1.evaluate(test_input_fn)

In [ ]:
results

In [ ]:
model_l2 = tf.estimator.LinearClassifier(feature_columns=all_columns, optimizer=tf.train.FtrlOptimizer(
    l1_regularization_strength=0.0, l2_regularization_strength=10.0, learning_rate=0.1))
model_l2

In [ ]:
model_l2.train(train_input_fn)
results = model_l2.evaluate(test_input_fn)

In [ ]:
results

It is not much better than the base model.

In [ ]:
def get_flat_weights(model):
    
    weight_names =  [name for name in model.get_variable_names() if "linear_model" in name and "Ftrl" not in name]
    weight_values = [model.get_variable_value(name) for name in weight_names]
    
    weights_flat = np.concatenate([item.flatten() for item in weight_values], axis=0)
    return weights_flat

In [ ]:
weights_flat = get_flat_weights(model)
weights_flat_l1 = get_flat_weights(model_l1)
weights_flat_l2 = get_flat_weights(model_l2)

In [ ]:
weight_mask = weights_flat != 0

weights_base = weights_flat[weight_mask]
weights_l1 = weights_flat_l1[weight_mask]
weights_l2 = weights_flat_l2[weight_mask]

In [ ]:
plt.figure()
_ = plt.hist(weights_base, bins=np.linspace(-3,3,30))
plt.title('Base Model')
plt.ylim([0,500])

plt.figure()
_ = plt.hist(weights_l1, bins=np.linspace(-3,3,30))
plt.title('L1 - Regularization')
plt.ylim([0,500])

plt.figure()
_ = plt.hist(weights_l2, bins=np.linspace(-3,3,30))
plt.title('L2 - Regularization')
_=plt.ylim([0,500])